In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:

In [5]:
import chromadb
# For persistent storage
client = chromadb.PersistentClient(path="/content/chroma_db/")
# For in-memory storage
# client = chromadb.EphemeralClient()


In [6]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.3 MB/s eta 0:00:00


In [8]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text.append(text)
    return all_text

pdf_path = "your_textbook.pdf"
text_pages = extract_text_from_pdf("/content/healthy-pregnancy-guide-20_.pdf")


In [10]:
def preprocess_text(text):
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

processed_pages = [preprocess_text(page) for page in text_pages]


In [12]:
collection = client.get_or_create_collection(name="PregnancyHealth")


In [16]:
ids = [f"id{i}" for i in range(len(processed_pages))]  # Create a list of IDs
metadatas = [{"page_content": page} for page in processed_pages]  # Create a list of metadata dictionaries
collection.add(
    documents=processed_pages,
    metadatas=metadatas,
    ids=ids
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 33.5MiB/s]


In [18]:
import shutil
import zipfile

# Zip the directory
shutil.make_archive('/content/chroma_db_backup', 'zip', '/content/chroma_db/')

# Download the zip file
from google.colab import files
files.download('/content/chroma_db_backup.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import zipfile
import chromadb
# Unzip the file
with zipfile.ZipFile('/content/chroma_db_backup.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/chroma_db/')

# Load the ChromaDB client
client = chromadb.PersistentClient(path="/content/chroma_db/")


In [10]:
# Load the database
collection = client.get_collection(name="PregnancyHealth")

# Query the collection or perform operations
results = collection.query(query_texts=["can i smoke?"], n_results=5)
print(results)


{'ids': [['id43', 'id42', 'id45', 'id227', 'id44']], 'embeddings': None, 'documents': [['Why should | quit smoking? Smoking is the leading cause of preventable death in the United States. By quitting you can: ¢ Prolong your life. ¢ Lower your risk of heart disease. e Lower your risk of developing lung, throat, mouth, pancreatic and bladder cancer. e Lower your risk of developing breathing problems, such as chronic obstructive pulmonary disease (COPD), asthma and emphysema. e Lower your risk of developing allergies. e Raise your energy level. e Improve your appearance. Your skin will wrinkle less and look better, and your fingers and teeth will not be yellow. ¢ Improve your sense of smell and taste. ¢ Feel healthier overall, with improved self-esteem. * Save a lot of money (the average smoker spends $2,000 a year for cigarettes). How can | quit smoking? There is no one way to quit smoking that works for everyone, since each person has different smoking habits. Here are some tips: e Hide